In [6]:
import wget
wget.download('http://www-nlp.stanford.edu/data/glove.840B.300d.zip')

'glove.840B.300d.zip'

In [2]:
import wget
wget.download('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')

'GoogleNews-vectors-negative300.bin.gz'

In [11]:
wget.download('http://qim.ec.quoracdn.net/quora_duplicate_questions.tsv')

'quora_duplicate_questions.tsv'

In [3]:
import nltk

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zarrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
import _pickle as cPickle
import pandas as pd
import numpy as np
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
stop_words = stopwords.words('english')

In [18]:
def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)

In [19]:
def norm_wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return norm_model.wmdistance(s1, s2)


In [48]:
def sent2vec(s):
    words = str(s).lower() #.decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

In [33]:
data = pd.read_csv('quora-question-pairs/train.csv')

In [34]:
data = data.drop(['id', 'qid1', 'qid2'], axis=1)
data.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [35]:
#Get the length of questions strings and calculate the difference
data['len_q1'] = data.question1.apply(lambda x: len(str(x)))
data['len_q2'] = data.question2.apply(lambda x: len(str(x)))
data['diff_len'] = data.len_q1 - data.len_q2

In [36]:
#Get the number of unique chatacters in each string, exluding the white space
data['len_char_q1'] = data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_char_q2'] = data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))


In [37]:
#Get the number of words in each string
data['len_word_q1'] = data.question1.apply(lambda x: len(str(x).split()))
data['len_word_q2'] = data.question2.apply(lambda x: len(str(x).split()))

In [38]:
#Find the number of common words in q1 and q2
data['common_words'] = data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)


In [39]:
data['fuzz_qratio'] = data.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_WRatio'] = data.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)

In [40]:
data['fuzz_partial_ratio'] = data.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)

In [41]:
data['fuzz_partial_token_set_ratio'] = data.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_partial_token_sort_ratio'] = data.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

In [42]:
data['fuzz_token_set_ratio'] = data.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_token_sort_ratio'] = data.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

In [43]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
data['wmd'] = data.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

In [45]:
norm_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
norm_model.init_sims(replace=True)
data['norm_wmd'] = data.apply(lambda x: norm_wmd(x['question1'], x['question2']), axis=1)

In [46]:
question1_vectors = np.zeros((data.shape[0], 300))
error_count = 0

In [49]:
for i, q in tqdm(enumerate(data.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

275it [00:02, 186.10it/s]/Users/zarrina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
404290it [05:19, 1265.83it/s]


In [50]:
question2_vectors  = np.zeros((data.shape[0], 300))
for i, q in tqdm(enumerate(data.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

145it [00:00, 1435.20it/s]/Users/zarrina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
404290it [03:43, 1812.20it/s]


In [51]:
data['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

/Users/zarrina/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [52]:
data['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

In [53]:
data['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

/Users/zarrina/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:853: RuntimeWarning: invalid value encountered in double_scalars
  dist = np.double(unequal_nonzero.sum()) / np.double(nonzero.sum())


In [54]:
data['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

In [55]:
data['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

In [56]:
data['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

In [57]:
data['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]


/Users/zarrina/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:1138: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()


In [58]:
data['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
data['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
data['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
data['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]


In [59]:
cPickle.dump(question1_vectors, open('q1_w2v.pkl', 'wb'), -1)
cPickle.dump(question2_vectors, open('q2_w2v.pkl', 'wb'), -1)



In [60]:
data.to_csv('quora_features.csv', index=False)

Model

In [93]:
from keras import optimizers
from keras import backend as K

from keras.callbacks import ModelCheckpoint

from keras.layers import Embedding, Input
from keras.layers import TimeDistributed, Lambda
from keras.layers import Convolution1D, GlobalMaxPooling1D

from keras.layers.advanced_activations import PReLU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, GRU

from keras.models import Model
from keras.preprocessing import sequence, text

from keras.utils import np_utils


In [64]:
data.head()

,question1,question2,is_duplicate,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,...,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,66,57,9,20,20,14,12,...,5.081614,1.0,94.023324,0.371408,0.168999,0.186557,0.031817,-0.091902,0.050416,0.337301
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,51,88,-37,21,29,8,13,...,14.195119,1.0,177.588090,1.012091,0.455910,0.592655,0.008735,0.094704,0.284010,-0.034444
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,73,59,14,25,24,14,10,...,9.055989,1.0,135.988707,0.666346,0.307828,0.342306,0.239752,0.144554,0.026759,-0.474131
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,50,65,-15,19,26,11,9,...,15.987437,1.0,192.237828,1.140536,0.506028,0.692421,-0.002527,0.069649,-0.244560,-0.265568
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,76,39,37,25,18,13,7,...,12.103178,1.0,161.408435,0.860225,0.382770,0.480633,-0.133849,0.114777,0.217900,-0.338876


In [122]:
y = data.is_duplicate.values

tk = text.Tokenizer(num_words=200000)

max_len = 40
tk.fit_on_texts(list(data.question1.values.astype(str)) + list(data.question2.values.astype(str)))
x1 = tk.texts_to_sequences(data.question1.values.astype(str))
x1 = sequence.pad_sequences(x1, maxlen=max_len)

x2 = tk.texts_to_sequences(data.question2.values.astype(str))
x2 = sequence.pad_sequences(x2, maxlen=max_len)

word_index = tk.word_index

ytrain_enc = np_utils.to_categorical(y)

In [82]:
embeddings_index = {}
f = open('glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [04:45, 7696.27it/s] 


Found 2196016 word vectors.


In [83]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


100%|██████████| 95596/95596 [06:26<00:00, 247.55it/s] 


In [ ]:
max_features = 200000
filter_length = 5
num_filter = 64
pool_length = 4

input_1 = Input(shape=(40,))
embedding_1 = Embedding(input_dim=len(word_index) + 1,
                        output_dim=300,
                        weights=[embedding_matrix],
                        input_length=40,
                        trainable=False)(input_1)

timedistributed_1 = TimeDistributed(Dense(300, activation='relu'))(embedding_1)
lambda_1 = Lambda(lambda x: K.sum(x, axis=1), output_shape=(300,))(timedistributed_1)


input_2 = Input(shape=(40,))
embedding_2 = Embedding(input_dim=len(word_index) + 1,
                        output_dim=300,
                        weights=[embedding_matrix],
                        input_length=40,
                        trainable=False)(input_2)

timedistributed_2 = TimeDistributed(Dense(300, activation='relu'))(embedding_1)
lambda_2 = Lambda(lambda x: K.sum(x, axis=1), output_shape=(300,))(timedistributed_1)



input_3 = Input(shape=(40,))
embedding_3 = Embedding(input_dim=len(word_index) + 1,
                        output_dim=300,
                        weights=[embedding_matrix],
                        input_length=40,
                        trainable=False)(input_3)

convolution_3 = Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1)(embedding_3)

dropout_3 = Dropout(0.2)(convolution_3)
convolution_3_2 = Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1)(dropout_3)


globalmaxpooling1d_3 = GlobalMaxPooling1D()(convolution_3_2)

dropout_3_2 = Dropout(0.2)(globalmaxpooling1d_3)

dense_3 = Dense(300)(globalmaxpooling1d_3)

dropout_3_3 = Dropout(0.2)(dense_3)
batchnormalization_3 = BatchNormalization()(dropout_3_3)


input_4 = Input(shape=(40,))
embedding_4 = Embedding(input_dim=len(word_index) + 1,
                        output_dim=300,
                        weights=[embedding_matrix],
                        input_length=40,
                        trainable=False)(input_4)

convolution_4 = Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1)(embedding_4)

dropout_4 = Dropout(0.2)(convolution_4)
convolution_4_2 = Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1)(dropout_4)


globalmaxpooling1d_4 = GlobalMaxPooling1D()(convolution_4_2)

dropout_4_2 = Dropout(0.2)(globalmaxpooling1d_4)

dense_4 = Dense(300)(globalmaxpooling1d_4)

dropout_4_3 = Dropout(0.2)(dense_4)
batchnormalization_4 = BatchNormalization()(dropout_4_3)

input_5 = Input(shape=(40,))
embedding_5 = Embedding(input_dim=len(word_index) + 1,
                        output_dim=300,
                       input_length=40)(input_5)

dropout_5 = Dropout(0.2)(embedding_5)
ltsm_5 = LSTM(300)(dropout_5)
dropout_5_2 = Dropout(0.2)(ltsm_5)

input_6 = Input(shape=(40,))
embedding_6 = Embedding(input_dim=len(word_index) + 1,
                        output_dim=300,
                       input_length=40)(input_6)

dropout_6 = Dropout(0.2)(embedding_6)
ltsm_6 = LSTM(300)(dropout_6)
dropout_6_2 = Dropout(0.2)(ltsm_6)


merged = concatenate([lambda_1, 
                      lambda_2, 
                      batchnormalization_3, 
                      batchnormalization_4,
                      dropout_5_2,
                      dropout_6_2])

m_dense = Dense(300)(merged)
m_relu = PReLU()(m_dense)
m_dropout = Dropout(0.2)(m_relu)
m_batch = BatchNormalization()(m_dropout)

m2_dense = Dense(300)(m_batch)
m2_relu = PReLU()(m2_dense)
m2_dropout = Dropout(0.2)(m2_relu)
m2_batch = BatchNormalization()(m2_dropout)


m3_dense = Dense(300)(m2_batch)
m3_relu = PReLU()(m3_dense)
m3_dropout = Dropout(0.2)(m3_relu)
m3_batch = BatchNormalization()(m3_dropout)

m4_dense = Dense(300)(m3_batch)
m4_relu = PReLU()(m4_dense)
m4_dropout = Dropout(0.2)(m4_relu)
m4_batch = BatchNormalization()(m4_dropout)

m5_dense = Dense(300)(m4_batch)
m5_relu = PReLU()(m_dense)
m5_dropout = Dropout(0.2)(m_relu)
m5_batch = BatchNormalization()(m5_dropout)


dense_out = Dense(1, activation='sigmoid')(m5_batch)

# build and compile model
model = Model(inputs=[input_1, 
                      input_2, 
                      input_3, 
                      input_4,
                      input_5, 
                      input_6], outputs=[dense_out])

model.compile(optimizers.Adam(), metrics=['accuracy'], loss='binary_crossentropy')

checkpoint = ModelCheckpoint('weights.h5', monitor='val_acc', save_best_only=True, verbose=2)
model.fit([x1, x2, x1, x2, x1, x2 ], y=y, batch_size=384, epochs=1,
                 verbose=1, validation_split=0.1, shuffle=True, callbacks=[checkpoint])